<a href="https://colab.research.google.com/github/lab-shima/lab/blob/main/%E3%83%87%E3%83%BC%E3%82%BF%E3%83%81%E3%82%A7%E3%83%83%E3%82%AF%E3%83%8E%E3%83%BC%E3%83%88%E3%83%96%E3%83%83%E3%82%AF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# データチェックノートブック

1.   統計量の出力
2.   テーブルのマージ＆両サイドの結合率
3.   

In [1]:
!pip install --upgrade deepl
import pandas as pd
import numpy as np
import deepl
from getpass import getpass

## グローバル変数
---

In [36]:
# 各ファイルのパスを指定
PATH = '/content/drive/MyDrive/Colab Notebooks/c01.csv'
AWARING_PATH = ''
SYNC_PATH = ''
BLUME_PATH = ''

# DeepL 変換のソース言語とターゲット言語を定義
SOURCE_LANG = 'JA'#'EN-US'
TARGET_LANG = 'EN-US'#'JA'

# DeepL APIキー取得
API_KEY = getpass(prompt="API キーを入力してください: ")


## 1 データ取得＆加工（ヘッダー和訳）
---

### 1.1 CSVデータの読み込み

In [85]:
# CSVファイルを読み込み
df_eng = pd.read_csv(PATH)
display(df_eng)

,都道府県コード,都道府県名,元号,和暦（年）,西暦（年）,注,人口（総数）,人口（男）,人口（女）
0,0,全国,大正,9,1920,NaN,55963053,28044185,27918868
1,1,北海道,大正,9,1920,NaN,2359183,1244322,1114861
2,2,青森県,大正,9,1920,NaN,756454,381293,375161
3,3,岩手県,大正,9,1920,NaN,845540,421069,424471
4,4,宮城県,大正,9,1920,NaN,961768,485309,476459
...,...,...,...,...,...,...,...,...,...
975,43,熊本県,平成,27,2015,NaN,1786170,841046,945124
976,44,大分県,平成,27,2015,NaN,1166338,551932,614406
977,45,宮崎県,平成,27,2015,NaN,1104069,519242,584827
978,46,鹿児島県,平成,27,2015,NaN,1648177,773061,875116


### 1.2 DeepL APIでヘッダーを翻訳

In [90]:
# df_engのヘッダーをリスト形式で取得
headers = df_eng.columns.tolist()

# イニシャライズ
translator = deepl.Translator(API_KEY)

# 翻訳を実行
translation_results = translator.translate_text(headers, source_lang=SOURCE_LANG, target_lang=TARGET_LANG)

# 翻訳結果は、'deepl.api_data.TextResultオブジェクト'形式で格納されているため、リスト形式に変換
translation_results_list = [translation_result.text for translation_result in translation_results]

# df_engをコピーしてdf_jpを作成
df_jp = pd.DataFrame(df_eng)

# df_jpのヘッダーを和訳結果に差し替え
df_jp.columns = translation_results_list

display(df_jp.head())

,prefecture code,prefecture name,era name,Japanese calendar (year),Year,annotation,Population (total),Population (Male),Population (Female)
0,0,全国,大正,9,1920,NaN,55963053,28044185,27918868
1,1,北海道,大正,9,1920,NaN,2359183,1244322,1114861
2,2,青森県,大正,9,1920,NaN,756454,381293,375161
3,3,岩手県,大正,9,1920,NaN,845540,421069,424471
4,4,宮城県,大正,9,1920,NaN,961768,485309,476459


## 2 データのチェック
---

### 2.1 各カラムの型チェック

In [91]:
# 各カラムのデータ型を一括表示
print(df_jp.dtypes)

prefecture code             object
prefecture name             object
era name                    object
Japanese calendar (year)     int64
Year                         int64
annotation                  object
Population (total)          object
Population (Male)           object
Population (Female)         object
dtype: object


### 2.2 レコード数／欠損数／欠損率／統計量のチェック
各指標の見方
*   count : 要素の数
*   unique : ユニークな要素数
*   top : 最頻値
*   freq : 最頻値の出現回数
*   unique_rate : ユニーク率
*   loss_rate : 欠損率




In [82]:
# データフレームの統計量を一括表示
describe_table = df_jp.describe(include='all')

# ユニーク率の計算
unique_rates = describe_table.loc['unique']/describe_table.loc['count']*100
unique_rates.name = 'unique_rate'

# 欠損率の計算
loss_rates = (len(df_jp) - describe_table.loc['count']) / len(df_jp)*100
loss_rates.name = 'loss_rates'  # 新しい行の名前を設定

# ユニーク率の計算
describe_table.loc['unique_rate'] = unique_rates
describe_table.loc['loss_rate'] = loss_rates

# descriptionデータフレームに欠損率の行を追加
display(describe_table)


,prefecture code,prefecture name,era name,Japanese calendar (year),Year,annotation,Population (total),Population (Male),Population (Female)
count,980,980,980,980.000000,980.000000,96,980,980,980
unique,50,50,3,NaN,NaN,2,979,980,979
top,0,全国,昭和,NaN,NaN,1),832832,28044185,397509
freq,20,20,584,NaN,NaN,48,2,1,2
mean,NaN,NaN,NaN,25.095918,1968.010204,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,17.012192,28.841611,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,2.000000,1920.000000,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,12.000000,1945.000000,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,22.000000,1970.000000,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,40.000000,1995.000000,NaN,NaN,NaN,NaN


In [100]:
# csvファイル読み込み、ヘッダーの翻訳、テーブル情報のチェック関数
def test(path, SOURCE_LANG, TARGET_LANG):
  # CSVファイルを読み込み
  df_eng = pd.read_csv(path)
  display(df_eng.head())

  # df_engのヘッダーをリスト形式で取得
  headers = df_eng.columns.tolist()

  # イニシャライズ
  translator = deepl.Translator(API_KEY)

  # 翻訳を実行
  translation_results = translator.translate_text(headers, source_lang=SOURCE_LANG, target_lang=TARGET_LANG)

  # 翻訳結果は、'deepl.api_data.TextResultオブジェクト'形式で格納されているため、リスト形式に変換
  translation_results_list = [translation_result.text for translation_result in translation_results]

  # df_engをコピーしてdf_jpを作成
  df_jp = pd.DataFrame(df_eng)

  # df_jpのヘッダーを和訳結果に差し替え
  df_jp.columns = translation_results_list

  display(df_jp.head())

  # シェイプの表示
  print('取得したテーブルの行・列数：',df_jp.shape)

  # 各カラムのデータ型を一括表示
  print(df_jp.dtypes)

  # データフレームの統計量を一括表示
  describe_table = df_jp.describe(include='all')

  # ユニーク率の計算
  unique_rates = describe_table.loc['unique']/describe_table.loc['count']*100
  unique_rates.name = 'unique_rate'

  # 欠損率の計算
  loss_rates = (len(df_jp) - describe_table.loc['count']) / len(df_jp)*100
  loss_rates.name = 'loss_rates'  # 新しい行の名前を設定

  # ユニーク率の計算
  describe_table.loc['unique_rate'] = unique_rates
  describe_table.loc['loss_rate'] = loss_rates

  # descriptionデータフレームに欠損率の行を追加
  display(describe_table)

In [101]:
test(PATH, SOURCE_LANG, TARGET_LANG)

,都道府県コード,都道府県名,元号,和暦（年）,西暦（年）,注,人口（総数）,人口（男）,人口（女）
0,0,全国,大正,9,1920,NaN,55963053,28044185,27918868
1,1,北海道,大正,9,1920,NaN,2359183,1244322,1114861
2,2,青森県,大正,9,1920,NaN,756454,381293,375161
3,3,岩手県,大正,9,1920,NaN,845540,421069,424471
4,4,宮城県,大正,9,1920,NaN,961768,485309,476459


,prefecture code,prefecture name,era name,Japanese calendar (year),Year,annotation,Population (total),Population (Male),Population (Female)
0,0,全国,大正,9,1920,NaN,55963053,28044185,27918868
1,1,北海道,大正,9,1920,NaN,2359183,1244322,1114861
2,2,青森県,大正,9,1920,NaN,756454,381293,375161
3,3,岩手県,大正,9,1920,NaN,845540,421069,424471
4,4,宮城県,大正,9,1920,NaN,961768,485309,476459


取得したテーブルの行・列数： (980, 9)
prefecture code             object
prefecture name             object
era name                    object
Japanese calendar (year)     int64
Year                         int64
annotation                  object
Population (total)          object
Population (Male)           object
Population (Female)         object
dtype: object


,prefecture code,prefecture name,era name,Japanese calendar (year),Year,annotation,Population (total),Population (Male),Population (Female)
count,980,980,980,980.000000,980.000000,96,980,980,980
unique,50,50,3,NaN,NaN,2,979,980,979
top,0,全国,昭和,NaN,NaN,1),832832,28044185,397509
freq,20,20,584,NaN,NaN,48,2,1,2
mean,NaN,NaN,NaN,25.095918,1968.010204,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,17.012192,28.841611,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,2.000000,1920.000000,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,12.000000,1945.000000,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,22.000000,1970.000000,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,40.000000,1995.000000,NaN,NaN,NaN,NaN


## 3 Merge Key 調査
---

### 3.1 結合率チェック

In [ ]:
data = df.merge(mst_process_kbn, on = 'process', how = '')